In [1]:
# %matplotlib notebook
%matplotlib notebook
import matplotlib.pyplot as plt
import xml.etree.cElementTree as ET
import numpy as np
from scipy.misc import toimage
from fuzzywuzzy import process


In [2]:
filename = 'AR-3-4-247' # you need a NXML file, plus an XML file (the latter generated from pdftoxml) with this base name

In [3]:
# get the basic elements from the pdftoxml XML file

data = []
page_meta = []
text_boxes = []
tokens = []

for event, elem in ET.iterparse('{}.xml'.format(filename), events=("start", "end")):

    if event=='start':
        if elem.tag=='PAGE':
            data.append({"page": elem.attrib,
                        "text": []
                        })

        if elem.tag=='TEXT':
         
            data[-1]['text'].append({"meta":elem.attrib,
                                     "tokens": []})
        if elem.tag=='TOKEN' and elem.text:
            s = elem.text.strip()
            data[-1]['text'][-1]["tokens"].append(s.replace('–', '-'))
    

In [4]:
# get the labelled text elements from the PMC nxml file
import html
table_snippets = set()

for event, elem in ET.iterparse('{}.nxml'.format(filename)):
    
    if elem.tag=='td' and elem.text:
        table_snippets.add(html.unescape(elem.text).strip())
        
title_snippets = set()
path = []
for event, elem in ET.iterparse('{}.nxml'.format(filename), events=["start", "end"]):
    if event == 'start':
        path.append(elem.tag)
    else:
        if elem.tag=='article-title' and elem.text and 'article-meta' in path:
            title_snippets.add(html.unescape(elem.text).strip())
        path.pop()
        

In [6]:
p = 4
pd = data[p-1]
pa = np.ones([int(pd['page']['height']), int(pd['page']['width'])])
for box in pd['text']:
    text_snippet = ' '.join(box['tokens']).strip()
    matches = process.extractOne(text_snippet, table_snippets)
    x, y, width, height = [int(float(box['meta'][i])) for i in ['x', 'y', 'width', 'height']]

    if matches and matches[1] > 90:
        col = 2
        pa[y:y+height,x:x+width] = col
    else:
        col = 0
        pa[y:y+height,x:x+width] = col
    
toimage(pa).show()    
    